In [3]:
import pytest
import torch

from typing import List

from flair.data import Sentence

def flair_sentence(tokens):

    sentences = [Sentence(i) for i in tokens]

    return sentences

In [4]:
flair_sentence([['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']])

[Sentence[9]: "The quick brown fox jumps over the lazy dog"]

In [6]:
from glirel.modules.token_rep import TokenRepLayer

token_rep = TokenRepLayer(
        model_name="microsoft/deberta-v3-small",
        fine_tune=True,
        subtoken_pooling="first",
        hidden_size=768,
        add_tokens=[],
    )

/Users/jesusherrera/miniconda3/envs/glirel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
token_rep = TokenRepLayer(
        model_name="microsoft/deberta-v3-small",
        fine_tune=True,
        subtoken_pooling="first",
        hidden_size=768,
        add_tokens=["[REL]", "[SEP]"],
    )

/Users/jesusherrera/miniconda3/envs/glirel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
tokens_batch = [
    ["Hello", "world"],                 
    ["This", "is", "a", "test"],        
]
lengths = torch.tensor([len(seq) for seq in tokens_batch])  

output = token_rep(tokens_batch, lengths)

In [8]:
output

{'embeddings': tensor([[[-0.8532, -0.1729, -0.1481,  ...,  0.2372,  0.0825, -0.3372],
          [ 1.0145, -0.7746, -0.5060,  ..., -0.0447, -0.0466, -0.3296],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-1.8800, -0.1707, -0.3937,  ...,  0.0134,  0.2015,  0.1256],
          [-1.9091, -0.0522,  0.2174,  ..., -0.0082,  0.3590,  0.0646],
          [-0.1139,  0.1377,  0.2996,  ...,  0.1373,  0.6843,  0.0982],
          [ 0.6009, -0.5112, -0.0265,  ...,  0.5871,  0.5133, -0.2278]]]),
 'mask': tensor([[1, 1, 0, 0],
         [1, 1, 1, 1]])}